<div align='center'>
    <h1>AutoML Tutorial</h1>
    <img src='https://github.com/vopani/fortyone/blob/main/images/automl_banner_530_x_455.png?raw=true'>
</div>

**Auto**mated **M**achine **L**earning (**AutoML**) has become widely adopted for building, experimenting and productionizing various types of machine learning models across business use-cases.

There are different open source solutions available and this notebook explores a simple baseline solution for some of them on the [Kaggle TPS (April 2021) competition](https://www.kaggle.com/c/tabular-playground-series-apr-2021).

* [AutoGluon](#AutoGluon)
* [EvalML](#EvalML)
* [FLAML](#FLAML)
* [H2O AutoML](#H2O-AutoML)
* [LightAutoML](#LightAutoML)
* [MLJAR](#MLJAR)
* [TPOT](#TPOT)

In [ ]:
## define configuration
PATH_TRAIN = '../input/tabular-playground-series-apr-2021/train.csv'
PATH_TEST = '../input/tabular-playground-series-apr-2021/test.csv'

PATH_AUTOGLUON_SUBMISSION = 'submission_autogluon.csv'
PATH_EVALML_SUBMISSION = 'submission_evalml.csv'
PATH_FLAML_SUBMISSION = 'submission_flaml.csv'
PATH_H2OAML_SUBMISSION = 'submission_h2oaml.csv'
PATH_LAML_SUBMISSION = 'submission_laml.csv'
PATH_MLJAR_SUBMISSION = 'submission_mljar.csv'
PATH_TPOT_SUBMISSION = 'submission_tpot.csv'

MAX_MODEL_RUNTIME_MINS = 10
MAX_MODEL_RUNTIME_SECS = MAX_MODEL_RUNTIME_MINS * 60

In [ ]:
## prepare data
import gc
import os
import shutil
import datatable as dt
from pathlib import Path

train = dt.fread(PATH_TRAIN)
test = dt.fread(PATH_TEST)

target = train['Survived'].to_numpy().ravel()
test_ids = test['PassengerId']

del train[:, ['PassengerId', 'Survived']]
test = test[:, train.names]

## AutoGluon
<img src='https://user-images.githubusercontent.com/16392542/77208906-224aa500-6aba-11ea-96bd-e81806074030.png' width='250px'>

[AutoGluon](https://auto.gluon.ai/stable/index.html) is an automl library open sourced by [Amazon](http://amazon.com/aws)

In [ ]:
## install packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q -U graphviz
!python3 -m pip install -q -U scikit-learn

In [ ]:
## import packages
from autogluon.tabular import TabularPredictor

In [ ]:
## run model
train['target'] = target

model_autogluon = TabularPredictor(label='target')
model_autogluon.fit(train_data=train.to_pandas(), excluded_model_types=['KNN'], time_limit=MAX_MODEL_RUNTIME_SECS)

del train['target']

In [ ]:
## check leaderboard
model_autogluon.leaderboard()

In [ ]:
## generate predictions
preds_autogluon = model_autogluon.predict_proba(test.to_pandas())[True]

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=dt.Frame(preds_autogluon))
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_AUTOGLUON_SUBMISSION)

In [ ]:
## clear memory
shutil.rmtree('AutogluonModels')
del model_autogluon

gc.collect()

Read more in [Documentation of AutoGluon](https://auto.gluon.ai/stable/index.html)

## EvalML
<img src='https://evalml.alteryx.com/en/stable/_images/evalml_horizontal.svg' width='250px'>

[EvalML](https://evalml.alteryx.com/en/stable) is an automl library open sourced by [Alteryx](http://www.alteryx.com)

In [ ]:
## install packages
!python3 -m pip install -q evalml==0.28.0

In [ ]:
## import packages
from evalml.automl import AutoMLSearch

In [ ]:
## run model
model_evalml = AutoMLSearch(X_train=train.to_pandas(), y_train=target, problem_type='binary', max_time=MAX_MODEL_RUNTIME_SECS)
model_evalml.search()

In [ ]:
## check leaderboard
model_evalml.rankings

In [ ]:
## generate predictions
preds_evalml = model_evalml.best_pipeline.predict_proba(test.to_pandas())[True]

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=dt.Frame(preds_evalml))
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_EVALML_SUBMISSION)

In [ ]:
## clear memory
os.remove('evalml_debug.log')
del model_evalml

gc.collect()

Read more in [Documentation of EvalML](https://evalml.alteryx.com)

## FLAML
<img src='https://github.com/microsoft/FLAML/raw/main/docs/images/FLAML.png' width='150px'>

[FLAML](https://microsoft.github.io/FLAML) is a fast and light automl library open sourced by [Microsoft](https://opensource.microsoft.com)

In [ ]:
## install packages
!python3 -m pip install -q flaml==0.5.0
!python3 -m pip install -q -U graphviz
!python3 -m pip install -q -U scikit-learn

In [ ]:
## import packages
from flaml import AutoML

In [ ]:
## run model
model_flaml = AutoML()
model_flaml.fit(X_train=train.to_pandas(), y_train=target, metric='accuracy', time_budget=MAX_MODEL_RUNTIME_SECS)

In [ ]:
## generate predictions
preds_flaml = model_flaml.predict_proba(test.to_pandas())[:, 1]

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=preds_flaml)
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_FLAML_SUBMISSION)

In [ ]:
## clear memory
if Path('catboost_info').exists():
    shutil.rmtree('catboost_info')

os.remove('default.log')
del model_flaml

gc.collect()

Read more in [Documentation of FLAML](https://microsoft.github.io/FLAML)

## H2O AutoML
<img src='https://docs.h2o.ai/h2o/latest-stable/h2o-docs/_images/h2o-automl-logo.jpg' width='150px'>

[H2O AutoML](https://www.h2o.ai/products/h2o-automl) is an automated machine learning library open sourced by [H2O.ai](https://h2o.ai)

In [ ]:
## import packages
import h2o
from h2o.automl import H2OAutoML

In [ ]:
## prepare data
h2o.init()

h2o_train = h2o.H2OFrame(train.to_pandas())
h2o_test = h2o.H2OFrame(test.to_pandas())

h2o_train['target'] = h2o.H2OFrame(target).asfactor()

In [ ]:
## run model
features = [x for x in h2o_train.columns if x != 'target']

model_h2oaml = H2OAutoML(stopping_metric='misclassification', max_runtime_secs=MAX_MODEL_RUNTIME_SECS)
model_h2oaml.train(x=features, y='target', training_frame=h2o_train)

In [ ]:
## check leaderboard
model_h2oaml.leaderboard

In [ ]:
## generate predictions
preds_h2oaml = model_h2oaml.leader.predict(h2o_test).as_data_frame()['True']

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=dt.Frame(preds_h2oaml))
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_H2OAML_SUBMISSION)

In [ ]:
## clear memory
h2o.cluster().shutdown()
del features, model_h2oaml

gc.collect()

Read more in [Documentation of H2O AutoML](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html)

## LightAutoML
<img src='https://github.com/sberbank-ai-lab/LightAutoML/blob/master/imgs/LightAutoML_logo_small.png?raw=true' width='150px'>

[LightAutoML](https://github.com/sberbank-ai-lab/LightAutoML) is a framework for automatic classification and regression model creation open sourced by [Sberbank](https://www.sberbank.com) AI Lab.

In [ ]:
## install packages
!python3 -m pip install -q lightautoml
!python3 -m pip install -q -U torch
!python3 -m pip install -q -U torchvision

In [ ]:
## import packages
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [ ]:
## run model
train['target'] = target

model_laml = TabularAutoML(task=Task(name='binary', metric='accuracy'), timeout=MAX_MODEL_RUNTIME_SECS)
model_laml.fit_predict(train_data=train.to_pandas(), roles={'target': 'target'})

del train['target']

In [ ]:
## generate predictions
preds_laml = model_laml.predict(test.to_pandas()).data.ravel()

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=preds_laml)
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_LAML_SUBMISSION)

In [ ]:
## clear memory
if Path('catboost_info').exists():
    shutil.rmtree('catboost_info')

del model_laml

gc.collect()

Read more in [Documentation of LightAutoML](https://lightautoml.readthedocs.io/en/latest/index.html)

## MLJAR
<img src='https://mljar.com/images/logo/mljar_circle3.svg' width='150px'>

[MLJAR](https://mljar.com) is an automated machine learning tool for tabular data

In [ ]:
## install packages
!python3 -m pip install -q mljar-supervised
!python3 -m pip install -q -U graphviz

In [ ]:
## import packages
from supervised import AutoML

In [ ]:
## run model
model_mljar = AutoML(eval_metric='accuracy', total_time_limit=MAX_MODEL_RUNTIME_SECS, results_path='./model_mljar')
model_mljar.fit(X=train.to_pandas(), y=target)

In [ ]:
## check leaderboard
model_mljar.get_leaderboard()

In [ ]:
## generate predictions
preds_mljar = model_mljar.predict_proba(test.to_pandas())[:, 1]

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=preds_mljar)
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_MLJAR_SUBMISSION)

In [ ]:
## clear memory
shutil.rmtree('model_mljar')
del model_mljar

gc.collect()

Read more in [Documentation of MLJAR](https://supervised.mljar.com)

## TPOT
<img src='https://raw.githubusercontent.com/EpistasisLab/tpot/master/images/tpot-logo.jpg' width='150px'>

[TPOT](http://epistasislab.github.io/tpot) is a Python Automated Machine Learning tool that optimizes machine learning pipelines using genetic programming

In [ ]:
## import packages
from tpot import TPOTClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_test = dt.rbind(train, test)

for col in ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']:
    le = LabelEncoder()
    train_test[col] = le.fit_transform(train_test[col])

train_tpot = train_test[:train.nrows, :]
test_tpot = train_test[train.nrows:, :]

model_tpot = TPOTClassifier(scoring='roc_auc', n_jobs=-1, max_time_mins=MAX_MODEL_RUNTIME_MINS)
model_tpot.fit(features=train_tpot.to_pandas(), target=target)

In [ ]:
## check pipeline
print(model_tpot.fitted_pipeline_)

In [ ]:
## generate predictions
preds_tpot = model_tpot.predict_proba(test_tpot.to_pandas())[:, 1]

In [ ]:
## create submission
submission = dt.Frame(PassengerId=test_ids, Survived=preds_tpot)
submission.head()

In [ ]:
## save submission
submission.to_csv(PATH_TPOT_SUBMISSION)

In [ ]:
## clear memory
del train_tpot, test_tpot, model_tpot

gc.collect()

Read more in [Documentation of TPOT](http://epistasislab.github.io/tpot)

## Similar Tutorials
Similar tutorials on other Kaggle TPS competitions are published here:

* [AutoML Tutorial: TPS (January 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-january-2021)
* [AutoML Tutorial: TPS (February 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-february-2021)
* [AutoML Tutorial: TPS (March 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-march-2021)
* [AutoML Tutorial: TPS (May 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-may-2021)
* [AutoML Tutorial: TPS (June 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-june-2021)
* [AutoML Tutorial: TPS (July 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-july-2021)
* [AutoML Tutorial: TPS (August 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-august-2021)
* [AutoML Tutorial: TPS (September 2021)](https://www.kaggle.com/rohanrao/automl-tutorial-tps-september-2021)